In [2]:
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import tiktoken
import os

from graphrag.query.indexer_adapters import (
    read_indexer_communities,
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch
from httpx import request

load_dotenv()

C:\Anaconda3lenvs\golf_v8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## VLM setup

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
vlm_model = "chatgpt-4o-latest"

vlm = ChatOpenAI(
    api_key=api_key,
    model=vlm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.encoding_for_model(vlm_model)

## Load community reports as context for global search

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = "C:\Code\III_2024_golf\\ragtest\output"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [5]:
community_df = pd.read_parquet(f"{INPUT_DIR}/create_final_communities.parquet")
entity_df = pd.read_parquet(f"{INPUT_DIR}/create_final_entities.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/create_final_community_reports.parquet")
node_df = pd.read_parquet(f"{INPUT_DIR}/create_final_nodes.parquet")
report_df

,id,human_readable_id,community,parent,level,title,summary,full_content,rank,rank_explanation,findings,full_content_json,period,size
0,7f94ed38c3f945e3ad1ad19fa548a3ba,83,83,19,2,力量与速度结合原则与Wenger与Bell的研究,该社区围绕力量与速度结合原则及其理论基础展开，Wenger与Bell的研究为这一原则提供了重...,# 力量与速度结合原则与Wenger与Bell的研究\n\n该社区围绕力量与速度结合原则及其...,4.0,影响严重性评分为中等，主要由于力量与速度结合原则在高尔夫运动中的应用可能影响运动员的表现。,[{'explanation': '力量与速度结合原则是高尔夫挥杆中的关键概念，运动员需要合...,"{\n ""title"": ""力量与速度结合原则与Wenger与Bell的研究"",\n ...",2025-02-17,2
1,7768076af1e14a3e8a887a481ad52c3a,84,84,19,2,Golf Swing Mechanics and Muscle Dynamics,This community focuses on the key muscle group...,# Golf Swing Mechanics and Muscle Dynamics\n\n...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The community identifies sev...,"{\n ""title"": ""Golf Swing Mechanics and Musc...",2025-02-17,13
2,3159c82f269a4e9d917cb25520941adc,85,85,25,2,Golf Clubs and Their Components,The community focuses on various types of golf...,# Golf Clubs and Their Components\n\nThe commu...,6.5,The impact severity rating is moderate to high...,[{'explanation': 'The community encompasses a ...,"{\n ""title"": ""Golf Clubs and Their Componen...",2025-02-17,8
3,c92340cde4cd431b8b3f981b81f62d22,86,86,25,2,Amateur and Professional Golfers Community,The community consists of amateur and professi...,# Amateur and Professional Golfers Community\n...,4.0,The impact severity rating is moderate due to ...,[{'explanation': 'The community is characteriz...,"{\n ""title"": ""Amateur and Professional Golf...",2025-02-17,2
4,b254bb63d09940718b8141c9bb9bd601,87,87,32,2,Golf Swing Mechanics Research Community,The community is centered around the collabora...,# Golf Swing Mechanics Research Community\n\nT...,4.0,The impact severity rating is moderate due to ...,[{'explanation': 'DeDe and Linda have a strong...,"{\n ""title"": ""Golf Swing Mechanics Research...",2025-02-17,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,c37b97c25340485f92a87334bf5b8daa,9,9,-1,0,Golf Swing Techniques and Research Community,This community focuses on the various aspects ...,# Golf Swing Techniques and Research Community...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'Preparatory movements are cr...,"{\n ""title"": ""Golf Swing Techniques and Res...",2025-02-17,71
109,1c5881de6f7047b59ef1dfb97ebf3bc8,10,10,-1,0,Golf Swing Techniques and Research Community,This community focuses on the techniques and r...,# Golf Swing Techniques and Research Community...,7.5,The impact severity rating is high due to the ...,[{'explanation': '揮桿 is the fundamental action...,"{\n ""title"": ""Golf Swing Techniques and Res...",2025-02-17,47
110,40d27c3a053f46ad8316a7c6eebc08ab,11,11,-1,0,Golf Swing Techniques and Specialized Strength...,This community focuses on the intersection of ...,# Golf Swing Techniques and Specialized Streng...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The study of biomechanics is...,"{\n ""title"": ""Golf Swing Techniques and Spe...",2025-02-17,44
111,4e70b8064b8840a89cdfa6ecfbc0ff3a,12,12,-1,0,Golf Equipment Research Community,The community is centered around the research ...,# Golf Equipment Research Community\n\nThe com...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The design of the golf club ...,"{\n ""title"": ""Golf Equipment Research Commu...",2025-02-17,74


In [6]:

communities = read_indexer_communities(community_df,node_df, report_df)
reports = read_indexer_reports(report_df, node_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(node_df,entity_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)

report_df.head()

Total report count: 113
Report count after filtering by community level 2: 87


,id,human_readable_id,community,parent,level,title,summary,full_content,rank,rank_explanation,findings,full_content_json,period,size
0,7f94ed38c3f945e3ad1ad19fa548a3ba,83,83,19,2,力量与速度结合原则与Wenger与Bell的研究,该社区围绕力量与速度结合原则及其理论基础展开，Wenger与Bell的研究为这一原则提供了重...,# 力量与速度结合原则与Wenger与Bell的研究\n\n该社区围绕力量与速度结合原则及其...,4.0,影响严重性评分为中等，主要由于力量与速度结合原则在高尔夫运动中的应用可能影响运动员的表现。,[{'explanation': '力量与速度结合原则是高尔夫挥杆中的关键概念，运动员需要合...,"{\n ""title"": ""力量与速度结合原则与Wenger与Bell的研究"",\n ...",2025-02-17,2
1,7768076af1e14a3e8a887a481ad52c3a,84,84,19,2,Golf Swing Mechanics and Muscle Dynamics,This community focuses on the key muscle group...,# Golf Swing Mechanics and Muscle Dynamics\n\n...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The community identifies sev...,"{\n ""title"": ""Golf Swing Mechanics and Musc...",2025-02-17,13
2,3159c82f269a4e9d917cb25520941adc,85,85,25,2,Golf Clubs and Their Components,The community focuses on various types of golf...,# Golf Clubs and Their Components\n\nThe commu...,6.5,The impact severity rating is moderate to high...,[{'explanation': 'The community encompasses a ...,"{\n ""title"": ""Golf Clubs and Their Componen...",2025-02-17,8
3,c92340cde4cd431b8b3f981b81f62d22,86,86,25,2,Amateur and Professional Golfers Community,The community consists of amateur and professi...,# Amateur and Professional Golfers Community\n...,4.0,The impact severity rating is moderate due to ...,[{'explanation': 'The community is characteriz...,"{\n ""title"": ""Amateur and Professional Golf...",2025-02-17,2
4,b254bb63d09940718b8141c9bb9bd601,87,87,32,2,Golf Swing Mechanics Research Community,The community is centered around the collabora...,# Golf Swing Mechanics Research Community\n\nT...,4.0,The impact severity rating is moderate due to ...,[{'explanation': 'DeDe and Linda have a strong...,"{\n ""title"": ""Golf Swing Mechanics Research...",2025-02-17,2


# Build global context based on community reports

In [7]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    communities=communities,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

# Perform global search

In [8]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [9]:
search_engine = GlobalSearch(
    llm=vlm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

##　VLM分析結果由GraphRAG增強

In [26]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv

'''
setting
'''
# 載入環境變數
load_dotenv()

# 讀取 回饋規則 檔案
file_path = "C:\Code\III_2024_golf\data\回饋規則.xlsx"
data = pd.read_excel(file_path)

# 讀取初始提示
init_prompt =  open('C:\Code\III_2024_golf\data\init_prompt.txt').read()

# 將數據轉換為文字格式（或提取特定欄位）
text_representation = data.to_string()

# 定義提示字串
prompt = f"""
{init_prompt}
以下是一些來自回饋規則檔案的數據：
{text_representation}"""

'''
setting
'''


def call_openai_api(image, text):
    # 呼叫 OpenAI API
    client = OpenAI()

    completion = client.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=[
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text",
                     "text": text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image}",
                            "detail": "high",
                        }
                    },
                ],
            }
        ]
    )

    print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [27]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [40]:
init_prompt =  open('C:\Code\III_2024_golf\data\\rag_init_prompt.txt').read()
image = encode_image("C:\Code\III_2024_golf\data/combined/combined_199353.jpg")
query = """
Push Slice 右拉右曲球	NULL	
球速:72.111、發射角度:12.173、發射方向:6.402、飛行距離:168.8、ClubAngleFace:8.779、ClubAnglePath:-2.799
front:{A: [0.177, -0.089, 0.163, -0.122, -0.088, -0.044], F: [-0.009, -0.011, 0.0, 0.049, -0.018, -0.012], I: [-0.02, -0.04, 0.182, -0.062, -0.04, -0.023], T: [0.004, -0.001, 0.0, -0.002, -0.001, -0.001]}
side:{A: [0.0, -0.002, 0.005, -0.002, -0.002, -0.0], F: [0.001, -0.001, 0.0, -0.002, -0.001, 0.002], I: [0.055, -0.026, 0.008, -0.007, -0.019, -0.009], T: [0.0, -0.0, 0.0, -0.0, -0.0, 0.0]}
"""

In [50]:
import nest_asyncio
nest_asyncio.apply()#solve asyncio loop problem

# request = call_openai_api(image, query)
request = """
        "球路": "Push Slice右拉右曲球",
        "原因": "P5~6下桿角度過於平緩，左手腕過度外展，肩關節伸展抬起，腰部向前旋轉過度，身體重心太早向前移動",
        "建議": "下桿時，手腕及肩膀往左轉動，腰部減少旋轉"
"""

result = search_engine.search(
    "你是一位專業的高爾夫教練，請使用你的知識增強以下建議，並按照原格式輸出:"+request
)

print(result.response)

```json
{
    "球路": "Push Slice右拉右曲球",
    "原因": "P5~6下桿角度過於平緩，左手腕過度外展，肩關節伸展抬起，腰部向前旋轉過度，身體重心太早向前移動，導致球桿路徑過於向右，桿面相對於路徑開啟，使球向右飛行並產生右曲球",
    "建議": "下桿時，手腕及肩膀往左轉動，以確保桿面角度更穩定，減少右曲球的發生。同時，腰部應減少過度旋轉，確保下半身穩定，避免重心過早向前移動。此外，應調整下桿軌跡，使其更陡峭，以確保更穩定的擊球路徑，並透過練習延遲擊球技術來改善擊球精準度"
}
```


# 測試融入圖片

In [23]:
result = search_engine.search(
    init_prompt +image + query
)

print(result.response)#超過token限制

Exception in _map_response_single_batch
Traceback (most recent call last):
  File "C:\Anaconda3lenvs\golf_v8\lib\site-packages\graphrag\query\structured_search\global_search\search.py", line 232, in _map_response_single_batch
    search_response = await self.llm.agenerate(
  File "C:\Anaconda3lenvs\golf_v8\lib\site-packages\graphrag\query\llm\oai\chat_openai.py", line 142, in agenerate
    async for attempt in retryer:
  File "C:\Anaconda3lenvs\golf_v8\lib\site-packages\tenacity\asyncio\__init__.py", line 166, in __anext__
    do = await self.iter(retry_state=self._retry_state)
  File "C:\Anaconda3lenvs\golf_v8\lib\site-packages\tenacity\asyncio\__init__.py", line 153, in iter
    result = await action(retry_state)
  File "C:\Anaconda3lenvs\golf_v8\lib\site-packages\tenacity\_utils.py", line 99, in inner
    return call(*args, **kwargs)
  File "C:\Anaconda3lenvs\golf_v8\lib\site-packages\tenacity\__init__.py", line 418, in exc_check
    raise retry_exc.reraise()
  File "C:\Anaconda3len

I am sorry but I am unable to answer this question given the provided data.
